In [1]:
from data import data_generation, data_picker, batch_creator, gp_kernels
from helpers import metrics
import sklearn.gaussian_process as gp 
import matplotlib.pyplot as plt
import numpy as np
import scipy
plt.style.use('ggplot')

%matplotlib inline

In [2]:
extrapo = False

In [3]:
if extrapo:
    x = np.load('/Users/omernivron/Downloads/x_te_extrapo')
    y = np.load('/Users/omernivron/Downloads/y_te_extrapo')
else:
    x = np.load('/Users/omernivron/Downloads/x_tr_interpo.npy')
    y = np.load('/Users/omernivron/Downloads/y_tr_interpo.npy')

In [4]:
# # GP independent sequence learning  
# online = False; mse_metric = 0; r_sq_metric = 0; kuee_metric = 0
# μ = []; σ = []
# m = int(x.shape[0] / 10)
# y_mean = np.mean(y[:m, :40])
# y_te = y[:m, 40]
# for j in range(0, m):
#     λ=np.exp(-1); ν=1;
#     kernel = ν**2 * gp.kernels.RBF(length_scale=λ) #+ gp.kernels.WhiteKernel(noise_level=σ)
#     model = gp.GaussianProcessRegressor(kernel=kernel)
#     x_tr =  x[j, :40].reshape(-1, 1); y_tr = y[j, :40].reshape(-1, 1)
#     model.fit(x_tr, y_tr)   
#     if online:
#         pass
#     else:
#         μ_te, σ_te = model.predict(x[j, 40].reshape(-1, 1), return_std=True)
#         μ.append(μ_te); σ.append(σ_te)

#     kuee_metric += metrics.KUEE(y_te[j], μ_te, σ_te)
    
# mse_metric = metrics.mse(y_te, μ)
# r_sq_metric = metrics.r_squared(y_te, μ, y_mean)
# mse_metric *= (1 / m)
# kuee_metric *= (1 / m)

In [5]:

#     fig, axes = plt.subplots(1, 1, figsize=(8, 6))
#     axes.set_ylim(-1, 1)
#     axes.scatter(xx_tr, yy_tr, c = 'black')
#         axes.scatter(xx_te, yy_te, c='purple', label = 'true y')
#         axes.scatter(xx_te, μ_te, c='blue', label = 'predicted y')
#         axes.errorbar(xx_te, μ_te, yerr = [ 2 * σ_te], c ='navy')
#         axes.legend()

#         plt.show()
#         plt.close()

In [6]:
idx = np.concatenate((np.random.choice(np.arange(0, 40), size = (40000, 5) , replace = True),
                     np.random.choice(np.arange(40, 59), size = (40000, 1) , replace = True)), axis = 1)

In [7]:
rows = np.repeat(np.arange(0, 3000, 1), 6).reshape(3000, 6)

In [8]:
x = x[rows, idx[:3000, :]]; y = y[rows, idx[:3000, :]]

In [ ]:
# GP independent sequence learning  
online = False; mse_metric = 0; r_sq_metric = 0; kuee_metric = 0
m = int(x.shape[0])
y_mean = np.mean(y[:m, :5])
y_te = y[:m, 5]
λ=np.exp(-1); ν=1;
kernel = ν**2 * gp.kernels.RBF(length_scale=λ) #+ gp.kernels.WhiteKernel(noise_level=σ)
model = gp.GaussianProcessRegressor(kernel=kernel)
x_tr =  x[:, :5].reshape(-1, 1); y_tr = y[:, :5].reshape(-1, 1)
model.fit(x_tr, y_tr)   
μ_te, σ_te = model.predict(x[:, 5].reshape(-1, 1), return_std=True)
kuee_metric = metrics.KUEE(y_te, μ_te, σ_te)
mse_metric = metrics.mse(y_te, μ_te)
r_sq_metric = metrics.r_squared(y_te, μ_te, y_mean)
mse_metric *= (1 / m)
kuee_metric *= (1 / m)

In [25]:
y_te

array([-0.06702726,  0.8412372 , -2.50708972, ...,  0.69480657,
        0.88466094,  1.59990592])

In [27]:
r_sq_metric = metrics.r_squared(y_te, μ, y_mean)

In [28]:
(mse_metric)

5038.124560611033

In [29]:
kuee_metric

0.92825

In [30]:
r_sq_metric

-4923.88316170636